In [1]:
import pandas as pd

In [2]:
tweets = pd.read_csv("training.1600000.processed.noemoticon.csv", names=["target", "ids", "date", "flag", "user", "text"], encoding='latin-1')

In [3]:
tweets.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
def extract_mentions(text):
    mentions = [word[1:] for word in text.split() if word.startswith('@')]
    return ', '.join(mentions) if mentions else None

def extract_tags(text):
    tags = [word[1:] for word in text.split() if word.startswith('#')]
    return ', '.join(tags) if tags else None

def extract_links(text):
    links = [word for word in text.split() if word.startswith('http')]
    return ', '.join(links) if links else None

In [8]:
tweets['mentions'] = tweets['text'].apply(extract_mentions)
tweets['tags'] = tweets['text'].apply(extract_tags)
tweets['links'] = tweets['text'].apply(extract_links)

In [9]:
tweets.head()

,target,ids,date,flag,user,text,mentions,tags,links
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot,None,http://twitpic.com/2y1zl
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,None,None,None
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,Kenichan,None,None
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,None,None,None
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass,None,None


In [10]:
tweets = tweets.drop(columns=['date', 'flag','target'])

In [11]:
tweets.head()

,ids,user,text,mentions,tags,links
0,1467810369,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot,None,http://twitpic.com/2y1zl
1,1467810672,scotthamilton,is upset that he can't update his Facebook by ...,None,None,None
2,1467810917,mattycus,@Kenichan I dived many times for the ball. Man...,Kenichan,None,None
3,1467811184,ElleCTF,my whole body feels itchy and like its on fire,None,None,None
4,1467811193,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass,None,None


In [12]:
tweets_with_tags = tweets[tweets['tags'].notna()]

In [13]:
tweets_with_tags.head()

,ids,user,text,mentions,tags,links
83,1467834265,mike_webster_au,@markhardy1974 Me too #itm,markhardy1974,itm,None
175,1467856821,mydwynter,@machineplay I'm so sorry you're having to go ...,machineplay,therapyfail,None
190,1467859558,tchviolin,"I activated my Selfcontrol block early, meanin...",None,fb,None
193,1467859922,Karmatical,@goodlaura What about Reese dying on #TTSC? An...,goodlaura,"TTSC?, 24",None
331,1467895481,RedTheTrucker,My man crush Jake Peavy let me down #gayforpeavy,None,gayforpeavy,None


In [14]:
tweets_with_tags.shape

(36002, 6)

In [13]:
tweets_with_tags.to_csv("tweets_with_tags.csv")

CALL apoc.load.json("file:///tweets.json") YIELD value
CREATE (t:Tweet {id: value.tweet_id, tweet_id: value.ids, text: value.text})
WITH t, value
UNWIND value.mentions AS mention
MERGE (u:User {username: mention})
MERGE (t)-[:MENTIONS]->(u)
WITH t, value
UNWIND value.links AS link
MERGE (l:Link {url: link})
MERGE (t)-[:CONTAINS_LINK]->(l)